## Data Audit - Tweets

As the Twitter API allows retrieval of up to 7 days of tweet history, and @Apple hasn't posted any tweet in the last 7 days, we are going to look at tweets from @realDonaldTrump in this notebook. 
Additionally, there hasn't been any mention of @Apple in other user's tweets.

Let's get the data from BigQuery, since it's not that big.

In [4]:
import google.cloud.bigquery as bq
import pandas as pd

client = bq.Client()
sql = """
    SELECT * FROM `test-project-datalab-225214.AAPL.tweets_realdonaldtrump`
"""
tweets_data = client.query(sql).to_dataframe()

print("There are {} rows.".format(len(tweets_data)))
tweets_data.head()

There are 400 rows.


,name,location,retweet_count,hashtags,created_at,text,screen_name
0,Donald J. Trump,"Washington, DC",10273,[],Sun Apr 07 22:02:04 +0000 2019,....I am pleased to announce that Kevin McAlee...,realDonaldTrump
1,Donald J. Trump,"Washington, DC",10516,[],Sun Apr 07 22:02:03 +0000 2019,Secretary of Homeland Security Kirstjen Nielse...,realDonaldTrump
2,Donald J. Trump,"Washington, DC",7844,[],Sun Apr 07 19:54:53 +0000 2019,RT @mike_pence: .@RealDonaldTrump is doing his...,realDonaldTrump
3,Donald J. Trump,"Washington, DC",11164,[],Sun Apr 07 19:53:56 +0000 2019,"RT @mike_pence: The last 2 years, under @RealD...",realDonaldTrump
4,Donald J. Trump,"Washington, DC",14937,[],Sun Apr 07 19:50:54 +0000 2019,Pleased to report that the American tourist an...,realDonaldTrump


In [31]:
df = tweets_data.copy(deep=True)
#making sure to use the timestamp as index
df["created_at"] = pd.to_datetime(df["created_at"])
df.index = df["created_at"]
del df["created_at"]
df.head()

,name,location,retweet_count,hashtags,text,screen_name
created_at,,,,,,
2019-04-07 22:02:04+00:00,Donald J. Trump,"Washington, DC",10273,[],....I am pleased to announce that Kevin McAlee...,realDonaldTrump
2019-04-07 22:02:03+00:00,Donald J. Trump,"Washington, DC",10516,[],Secretary of Homeland Security Kirstjen Nielse...,realDonaldTrump
2019-04-07 19:54:53+00:00,Donald J. Trump,"Washington, DC",7844,[],RT @mike_pence: .@RealDonaldTrump is doing his...,realDonaldTrump
2019-04-07 19:53:56+00:00,Donald J. Trump,"Washington, DC",11164,[],"RT @mike_pence: The last 2 years, under @RealD...",realDonaldTrump
2019-04-07 19:50:54+00:00,Donald J. Trump,"Washington, DC",14937,[],Pleased to report that the American tourist an...,realDonaldTrump


Let's check if there are duplicate entries.

In [32]:
df = df[~df.index.duplicated(keep='first')]

In [33]:
len(df)

201

It looks like we had some duplicated data.

It looks like we don't have null values in our data

Let's see if there are tweets were hashtags were used

In [34]:
df['num_hashtags'] = df['hashtags'].str.count("/") + 1

Let's check all null/None values.

In [35]:
df.isna().sum()

name               0
location           0
retweet_count      0
hashtags           0
text               0
screen_name        0
num_hashtags     201
dtype: int64

It seems that Trump hasn't added any hashtags to his tweets for the last 7 days. We are just going to get rid of that field for now.

In [36]:
df = df.drop(['hashtags','num_hashtags'], axis=1)

Ok, so now that we got rid of duplicate data and unnecessary fields, let's do some analysis.

## Data Analysis - Tweets